In [1]:
import pandas as pd
from tqdm import tqdm
import gensim
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# os.chdir(r"D://Proposal_Defense//Simulations")
#from Utils.Script_utils import get_data_splits, first_LSTM_training, get_data_splits_old_algo
from joblib import Parallel, delayed
import sys
from scipy.stats import norm
import numpy as np
import psutil
    
ram_gb = int(np.round(psutil.virtual_memory().total / (1024. **3)))
print("Ram is " + str(ram_gb), flush = True)
import multiprocessing
use_cores = multiprocessing.cpu_count()-1


Ram is 188


In [2]:
use_cores

47

In [3]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# # 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
# tf.config.set_visible_devices([], 'GPU')

from tqdm.notebook import tqdm
tf.random.set_seed(seed_value)

No GPU found


In [4]:
import matplotlib.pyplot as plt

import pickle



In [5]:
model_cbow = gensim.models.word2vec.Word2Vec.load(r"..//Data_Generation//word2vec_sg")

In [6]:
lr = 1e-3
# from sklearn.model_selection import train_test_split


In [7]:
reps = 50

In [8]:
## generate 100 realizations of train valid and test
# catch = Parallel(n_jobs=15, verbose = 10, backend = "loky")(delayed(get_data_splits_old_algo)(data,  features,  i) for i in range(reps)) 

In [9]:
with open('..//Data_Generation//Data//train_valid_test_splits_50.pkl', 'rb') as f:
    catch = pickle.load(f)  

In [10]:
with open('..//Data_Generation//LSTM_Heavy_Dropout//heavy_dropout_first_lstm.pkl', 'rb') as f:
    first_lstm = pickle.load(f)  

    
with open('..//Data_Generation//LSTM_Heavy_Dropout//Heavy_Data_train_logits.pkl', 'rb') as f:
    catch_train_logits = (pickle.load(f))    

with open('..//Data_Generation//LSTM_Heavy_Dropout//Heavy_Data_valid_logits.pkl', 'rb') as f:
    catch_valid_logits = (pickle.load( f))      
    
with open('..//Data_Generation//LSTM_Heavy_Dropout//Heavy_Data_test_logits.pkl', 'rb') as f:
    catch_test_logits = (pickle.load( f))      
    

In [11]:
with open('..//Data_Generation//LSTM_Heavy_Dropout//heavy_dropout_second_lstm.pkl', 'rb') as f:
    second_lstm = pickle.load(f)  

In [12]:
h1, h2 = 16,16

## batch size
batch_size = 32

In [13]:
from scipy.stats import multivariate_normal as mvn


from scipy.stats import invgamma, norm


from tqdm.notebook import tqdm


import random

lr = 0.1


import numpy as np


import time

from scipy.stats import multivariate_normal as mvn



from scipy.stats import multivariate_normal as mvn


import matplotlib.pyplot as plt

from scipy.special import expit

In [14]:
lr = 0.1

In [15]:
import numpy as np

In [16]:
catch1 = second_lstm

In [17]:
var_targets = 0.0001
rate = 0.5
bnn_reps = 100

In [18]:
def attention_lstm_model(rate, training): 
    padding_vector = np.zeros((1, model_cbow.wv.vectors.shape[1]))
    unknown_vector = np.zeros((1, model_cbow.wv.vectors.shape[1]))
    weight_vectors = np.vstack((padding_vector, unknown_vector))
    weight_vectors = np.vstack((weight_vectors, model_cbow.wv.vectors))
    embedding_layer = tf.keras.layers.Embedding(len(weight_vectors),
                            weight_vectors.shape[1],
                            weights=[weight_vectors],
                            mask_zero = False,
                            trainable=False)

    
    vectorize_layer = tf.keras.layers.TextVectorization(
                     output_mode='int',
                     vocabulary=model_cbow.wv.index_to_key, 
                     standardize = None)
    
    input_layer = tf.keras.layers.Input(shape = (1,), dtype = tf.string)

    vectorize = vectorize_layer(input_layer)

    vectorize.shape

    emb_output = embedding_layer(vectorize)

    emb_output.shape
    
    # spatial_drop = tf.keras.layers.SpatialDropout1D(0.5)
    
    # spatial_out = spatial_drop(emb_output, training = training)

    lstm_layer = tf.keras.layers.LSTM(25, return_sequences = True, dropout = rate)

    lstm_output = lstm_layer(emb_output, training = training)

#     x_a = tf.keras.layers.Dense(lstm_output.get_shape()[-1]//2, kernel_initializer = 'glorot_uniform', activation="tanh", name="tanh_mlp")(lstm_output) 
    
    # x_a = tf.keras.layers.SpatialDropout1D(0.75)(lstm_output, training = training)
    
    x_a = tf.keras.layers.Dense(1, kernel_initializer = 'glorot_uniform', activation='linear', name="word-level_context", kernel_regularizer=tf.keras.regularizers.L2())(lstm_output)

    x_a = tf.keras.layers.Flatten()(x_a)

    att_out = tf.keras.layers.Activation('softmax')(x_a) 

    x_a2 = tf.keras.layers.RepeatVector(lstm_output.get_shape()[-1])(att_out)

    x_a2 = tf.keras.layers.Permute([2,1])(x_a2)

    out = tf.keras.layers.Multiply()([lstm_output,x_a2])
    
    out = tf.keras.layers.Lambda(lambda x : tf.math.reduce_sum(x, axis = 1), name='expectation_over_words')(out)
    
    dropout_layer = tf.keras.layers.Dropout(rate)(out, training = training)
    
    # out = dropout_layer(out)

    pred_head = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.L2())

    pred_output = pred_head(dropout_layer)

    model = tf.keras.models.Model(input_layer, pred_output)
    
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                 optimizer = tf.keras.optimizers.Adam(learning_rate = lr), 
                 metrics=tf.keras.metrics.BinaryAccuracy())
    
    return model

In [19]:
import time

In [20]:
from scipy.stats import multivariate_normal as mvn

In [21]:
# var_weights =6
# var_weights_vec = 4
# var_targets = 0.04

In [22]:
from scipy.stats import multivariate_normal as mvn

In [23]:
threshold = 25

In [24]:
# train_fudged_probs

In [25]:
import matplotlib.pyplot as plt

In [26]:
from scipy.special import expit

In [27]:
from datetime import datetime 

In [28]:
doc2vec_dbow = gensim.models.doc2vec.Doc2Vec.load(r"..//Data_Generation//doc2vec_dbow")

In [29]:
def rep_one(idx, inflation_factor = 0.2, cutoff = 600): 

#     from scipy.special import expit
    patience_smaller = 0
    patience_uns = 0
# patience_bigger = 0

#     best_train_acc = 0
#     best_valid_acc = 1000

#     best_valid_mae = 10
    
    best_train_width = 100

    
    # best_width_train = 100
    
    threshold_achieved = False

    best_coverage_train = 0
    
    train_valid_seqs = np.vstack((catch[idx][0], catch[idx][1]))
    # train_valid_labels = np.vstack((y_train, y_valid))
    
    test_seqs = catch[idx][2]

    
    start = datetime.now()
    
    base_model = tf.keras.models.load_model("..//Data_Generation//LSTM_Heavy_Dropout//Real_World_Second_LSTM_Models_Heavy_Dropout" + "//" + "Model_" + str(idx))
    ens_model = attention_lstm_model(rate, True)
    ens_model.set_weights(base_model.get_weights())
    
            
    all_preds1 = [np.array(ens_model(train_valid_seqs)) for i in (range(0, bnn_reps))]
    all_preds1 = np.hstack(all_preds1)

    all_preds2 = expit(all_preds1)
    all_preds3 = all_preds2.mean(1).reshape(-1,1)
        
        

    li = np.percentile(all_preds2, axis = 1, q = (2.5, 97.5))[0,:].reshape(-1,1)
            
    ui = np.percentile(all_preds2, axis = 1, q = (2.5, 97.5))[1,:].reshape(-1,1)
            
    width = ui - li
            
    avg_width_train = np.mean(width)
        
    interim = expit(np.vstack((catch_train_logits[idx], catch_valid_logits[idx])))
        
    train_mae = np.mean(np.abs(interim.ravel() - all_preds3.ravel()))
        
            
    temp = interim
            
    ind = (temp >= li) & (temp <= ui)
            
    coverage_train= np.mean(ind.ravel())  
            
            
    all_preds1 = [np.array(ens_model(test_seqs)) for i in (range(0, bnn_reps))]
    all_preds1 = np.hstack(all_preds1)
    all_preds2 = expit(all_preds1)
    all_preds3 = all_preds2.mean(1).reshape(-1,1)
            
    li = np.percentile(all_preds2, axis = 1, q = (2.5, 97.5))[0,:].reshape(-1,1)
            
    ui = np.percentile(all_preds2, axis = 1, q = (2.5, 97.5))[1,:].reshape(-1,1)
            
    width = ui - li
            
    avg_width = np.mean(width)
        
    catch_test_probs = expit(catch_test_logits[idx])
            
    ind_test = (catch_test_probs >= li) & (catch_test_probs <= ui)
               
 
    coverage_test = np.mean(ind_test.ravel())    
    
    test_mae = np.mean(np.abs(catch_test_probs.ravel() - all_preds3.ravel()))
            

    

    return coverage_train, coverage_test, avg_width_train, avg_width
        


In [30]:
catch1 = second_lstm

In [31]:
# catch1[0][0]

In [32]:
# catch_train_logits_second[0]

In [33]:
def expit(x):
    """Compute softmax values for each sets of scores in x."""
#     e_x = np.exp(x - np.max(x))
    return 1 / (1 + np.exp(-x))

In [34]:
from scipy.stats import gamma

In [35]:
# second_lstm[0]

In [36]:
cur_idx =30

In [37]:
# uns_iter_threshold = 30
# threshold = 20
# cutoff_threshold = 50

In [38]:
# var_targets = 0.0001
# rate = 0.5
# bnn_reps = 50

In [39]:
# coverage_train, coverage_test, avg_width_train, avg_width 

In [40]:
coverage_train, coverage_test, avg_width_train, avg_width  = rep_one(cur_idx)

In [41]:
# best_coverage, best_test_coverage, avg_width = rep_one(cur_idx, second_lstm[cur_idx][0], second_lstm[cur_idx][1], second_lstm[cur_idx][2])

In [42]:
# best_train_coverage, best_test_coverage, best_train_width, best_test_width, avg_lstm_weight, exit_iter, exit_status, mins_taken

In [43]:
# best_train_coverage, best_test_coverage, best_train_width, best_test_width

In [44]:
# best_initial_targets_softmax.shape[1]

In [45]:
# random.seed(42)
# random_range = random.sample(range(0,best_initial_targets_test.shape[1]), k = 20)

In [46]:
# random_range

In [47]:
# import arviz as az
# a = norm.rvs(size=10000)
# # from hpd import hpd_grid
# ci_95 = az.hdi(a, hdi_prob=0.95)
# intervals, x, y, modes = hpd_grid(a, alpha=0.05)

In [48]:
# expit(az.hdi(best_initial_targets_test[:, i, :], hdi_prob=0.95))

In [49]:
# .shape

In [50]:
# fig, axes = plt.subplots(5, 4, figsize=(10, 10))
# axes = axes.flatten()
# counter = 0
# for i in random_range: 
# #     plt.figure(figsize = (10, 10))
#     axes[counter].hist(best_initial_targets_test[:, i, :])
#     axes[counter].axvline(x= catch_test_logits[cur_idx][i], color = "red")
#     u, l = np.percentile(best_initial_targets_test[:, i, :], q = (2.5, 97.5))
#     axes[counter].axvline(x= u, color = "green")
#     axes[counter].axvline(x= l, color = "green")    
#     # ax[1].hist(soft_train[:, i, :][:,1])
#     # ax[1].axvline(x= X_train_logits_true[i][1], color = "red")
# #     ax[2].hist(soft_train[:, i, :][:,2])
# #     ax[2].axvline(x= X_train_logits_true[i][2], color = "red")    
#     counter += 1
#     # plt.show()

In [51]:
# fig, axes = plt.subplots(5, 4, figsize=(10, 10))
# axes = axes.flatten()
# counter = 0
# for i in random_range: 
# #     plt.figure(figsize = (10, 10))
#     axes[counter].hist(best_initial_targets_softmax[:, i, :])
#     axes[counter].axvline(x= catch_test_probs[cur_idx][i], color = "red")
#     u, l = np.percentile(best_initial_targets_softmax[:, i, :], q = (2.5, 97.5))
#     axes[counter].axvline(x= u, color = "green")
#     axes[counter].axvline(x= l, color = "green")
#     # ax[1].hist(soft_train[:, i, :][:,1])
#     # ax[1].axvline(x= X_train_logits_true[i][1], color = "red")
# #     ax[2].hist(soft_train[:, i, :][:,2])
# #     ax[2].axvline(x= X_train_logits_true[i][2], color = "red")    
#     counter += 1
#     # plt.show()

In [52]:
catch_coverages = Parallel(n_jobs=15, verbose = 10, backend = "loky")(delayed(rep_one)(i) for i in range(reps))

[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done   2 tasks      | elapsed:   16.6s
[Parallel(n_jobs=15)]: Done  11 tasks      | elapsed:   19.2s
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:   32.0s
[Parallel(n_jobs=15)]: Done  27 out of  50 | elapsed:   34.9s remaining:   29.8s
[Parallel(n_jobs=15)]: Done  33 out of  50 | elapsed:   45.2s remaining:   23.3s
[Parallel(n_jobs=15)]: Done  39 out of  50 | elapsed:   48.4s remaining:   13.6s
[Parallel(n_jobs=15)]: Done  45 out of  50 | elapsed:   51.3s remaining:    5.7s
[Parallel(n_jobs=15)]: Done  50 out of  50 | elapsed:  1.0min finished


In [53]:
# catch_coverages

In [54]:
check = pd.DataFrame(catch_coverages)

In [55]:
# check

In [56]:
check.columns = ["train_coverage", "test_coverage", "avg_ci_width_train", "avg_ci_width_test"]

In [57]:
# check.columns = ["train_coverage", "test_coverage", "avg_ci_width_train", "avg_ci_width_test", "avg_lstm_weight"]

In [58]:
# check

In [59]:
# es = check[["exit_status"]].value_counts().reset_index()

In [60]:
# es.columns = ["exit_status", "frequency"]

In [61]:
# es.to_csv("exit_status_MCD_LSTM_Doc2Vec_" + "rate_" + str(rate) + "_bnn_reps_" + str(bnn_reps) + ".csv", index = False)

In [62]:
# check = check.drop(["exit_status"],1)

In [63]:
check_mean = check.mean(0).reset_index()

In [64]:
check_mean.columns = ["metrics", "value"]

In [65]:
check_mean

,metrics,value
0,train_coverage,0.813333
1,test_coverage,0.842500
2,avg_ci_width_train,0.544398
3,avg_ci_width_test,0.554002


In [66]:
check_mean.to_csv("mean_metrics_MCD_Just_LSTM" + "rate" + str(rate) + "_bnn_reps_" + str(bnn_reps) +  ".csv", index = False)

In [67]:
check_std = check.std(0).reset_index()

In [68]:
check_std.columns = ["metrics", "value"]

In [69]:
check_std.to_csv("std_dev_metrics_MCD_Just_LSTM_" + "rate" + str(rate) + "_bnn_reps_" + str(bnn_reps)+  ".csv", index = False)

In [70]:
# check.std(0)/np.sqrt(reps)

In [71]:
# import plotly.express as px
# # df = px.data.tips()
# fig = px.histogram(check_melt, x="variable", color="value")
# fig.show()

In [72]:
# rep_one(0, second_lstm[0][0], second_lstm[0][1], second_lstm[0][2])

In [73]:
# rep_one(1, second_pred_train_logits[i], second_pred_valid_logits[i], second_pred_test_logits[i])

In [74]:
# rep_one(3, second_pred_train_logits[i], second_pred_valid_logits[i], second_pred_test_logits[i])

In [75]:
# catch_all = []

# for i in range(reps): 
#     catch_all.append(rep_one(i, second_pred_train_logits[i], second_pred_valid_logits[i], second_pred_test_logits[i])[0])

In [76]:
# catch_all

In [77]:
# np.mean(catch_all)

In [78]:
# np.std(catch_all)/np.sqrt(reps)

In [79]:
# catch_coverages = []
# for i in tqdm(range(reps)): 
#     best_coverage, best_train_acc, best_valid_acc, best_test_acc = rep_one(i, second_lstm[i][0], second_lstm[i][1], second_lstm[i][2])
#     print(best_coverage, best_train_acc, best_valid_acc, best_test_acc)
#     catch_coverages.append([best_coverage, best_train_acc, best_valid_acc, best_test_acc])